In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
sample_order = pd.read_csv("instacart/sample_order.csv")

In [3]:
sample_order = sample_order.sort_values(by=['user_id', 'order_number']).reset_index(drop=True)
sample_order

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,1374495,3,prior,1,1,14,NaN,9387.0,1.0,0.0,Granny Smith Apples,24.0,4.0,fresh fruits,produce
1,1374495,3,prior,1,1,14,NaN,17668.0,2.0,0.0,Unsweetened Chocolate Almond Breeze Almond Milk,91.0,16.0,soy lactosefree,dairy eggs
2,1374495,3,prior,1,1,14,NaN,15143.0,3.0,0.0,Blueberry Pint,24.0,4.0,fresh fruits,produce
3,1374495,3,prior,1,1,14,NaN,16797.0,4.0,0.0,Strawberries,24.0,4.0,fresh fruits,produce
4,1374495,3,prior,1,1,14,NaN,39190.0,5.0,0.0,Vanilla Unsweetened Almond Milk,91.0,16.0,soy lactosefree,dairy eggs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8246811,1716008,206205,train,4,1,16,10.0,42495.0,15.0,0.0,Pink Lady (Cripps) Apple,24.0,4.0,fresh fruits,produce
8246812,1716008,206205,train,4,1,16,10.0,10181.0,16.0,1.0,Classic Guacamole,67.0,20.0,fresh dips tapenades,deli
8246813,1716008,206205,train,4,1,16,10.0,1158.0,17.0,0.0,Mango Chunks,116.0,1.0,frozen produce,frozen
8246814,1716008,206205,train,4,1,16,10.0,29499.0,18.0,0.0,"Ghee, Original",76.0,6.0,indian foods,international


In [4]:
sample_order.groupby(['user_id'])['eval_set'].last().value_counts()

train    31806
test     18194
Name: eval_set, dtype: int64

## Cumsum of days since prior order
- used for creating the feature "days since last bought of this product"

In [5]:
orders = pd.read_csv("instacart/orders.csv")
orders = orders.sort_values(by=['user_id','order_number']).reset_index(drop=True)
orders.head(20)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [6]:
orders.groupby('user_id')['days_since_prior_order'].cumsum()

0            NaN
1           15.0
2           36.0
3           65.0
4           93.0
           ...  
3421078    155.0
3421079    185.0
3421080    203.0
3421081    210.0
3421082    240.0
Name: days_since_prior_order, Length: 3421083, dtype: float64

In [7]:
orders['days_cumsum'] = orders.groupby('user_id')['days_since_prior_order'].cumsum()

In [8]:
sample_order = sample_order.merge(orders[['order_id', 'days_cumsum']], how='left', on='order_id')
sample_order.head(30)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,days_cumsum
0,1374495,3,prior,1,1,14,NaN,9387.0,1.0,0.0,Granny Smith Apples,24.0,4.0,fresh fruits,produce,NaN
1,1374495,3,prior,1,1,14,NaN,17668.0,2.0,0.0,Unsweetened Chocolate Almond Breeze Almond Milk,91.0,16.0,soy lactosefree,dairy eggs,NaN
2,1374495,3,prior,1,1,14,NaN,15143.0,3.0,0.0,Blueberry Pint,24.0,4.0,fresh fruits,produce,NaN
3,1374495,3,prior,1,1,14,NaN,16797.0,4.0,0.0,Strawberries,24.0,4.0,fresh fruits,produce,NaN
4,1374495,3,prior,1,1,14,NaN,39190.0,5.0,0.0,Vanilla Unsweetened Almond Milk,91.0,16.0,soy lactosefree,dairy eggs,NaN
5,1374495,3,prior,1,1,14,NaN,47766.0,6.0,0.0,Organic Avocado,24.0,4.0,fresh fruits,produce,NaN
6,1374495,3,prior,1,1,14,NaN,21903.0,7.0,0.0,Organic Baby Spinach,123.0,4.0,packaged vegetables fruits,produce,NaN
7,1374495,3,prior,1,1,14,NaN,39922.0,8.0,0.0,Celery Sticks,123.0,4.0,packaged vegetables fruits,produce,NaN
8,1374495,3,prior,1,1,14,NaN,24810.0,9.0,0.0,Organic Lightly Salted Brown Rice Cakes,78.0,19.0,crackers,snacks,NaN
9,1374495,3,prior,1,1,14,NaN,32402.0,10.0,0.0,"Crackers, Crispy, Cheddar",78.0,19.0,crackers,snacks,NaN


In [9]:
# set the first order to have days_cumsum = 0
sample_order['days_cumsum'] = sample_order['days_cumsum'].fillna(0)

## Split prior, train, test

In [10]:
sample_order_test = sample_order[sample_order['eval_set']=='test']
sample_order_train = sample_order[sample_order['eval_set']=='train']
sample_order_prior = sample_order[sample_order['eval_set']=='prior']

In [11]:
sample_order_prior.describe()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,aisle_id,department_id,days_cumsum
count,7.889718e+06,7.889718e+06,7.889718e+06,7.889718e+06,7.889718e+06,7.385777e+06,7.889718e+06,7.889718e+06,7.889718e+06,7.889718e+06,7.889718e+06,7.889718e+06
mean,1.708635e+06,1.033913e+05,1.702371e+01,2.730074e+00,1.341255e+01,1.112729e+01,2.557857e+04,8.358219e+00,5.903843e-01,7.123248e+01,9.928046e+00,1.237188e+02
std,9.877101e+05,5.978487e+04,1.734933e+01,2.085583e+00,4.246393e+00,8.791116e+00,1.409077e+04,7.119127e+00,4.917629e-01,3.820952e+01,6.280409e+00,9.662032e+01
min,3.000000e+00,3.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,8.528170e+05,5.146700e+04,5.000000e+00,1.000000e+00,1.000000e+01,5.000000e+00,1.353500e+04,3.000000e+00,0.000000e+00,3.100000e+01,4.000000e+00,4.100000e+01
50%,1.708149e+06,1.034330e+05,1.100000e+01,3.000000e+00,1.300000e+01,8.000000e+00,2.523700e+04,6.000000e+00,1.000000e+00,8.300000e+01,9.000000e+00,1.030000e+02
75%,2.564160e+06,1.554300e+05,2.400000e+01,5.000000e+00,1.600000e+01,1.500000e+01,3.791700e+04,1.100000e+01,1.000000e+00,1.070000e+02,1.600000e+01,1.940000e+02
max,3.421080e+06,2.062050e+05,9.900000e+01,6.000000e+00,2.300000e+01,3.000000e+01,4.968800e+04,1.450000e+02,1.000000e+00,1.340000e+02,2.100000e+01,3.650000e+02


In [12]:
# user_id = 3 bought product_id = 1819 multiple times
sample_order_prior = sample_order_prior.sort_values(by=['user_id', 'product_id', 'order_number']).reset_index(drop=True)
sample_order_prior

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,days_cumsum
0,444309,3,prior,2,3,19,9.0,248.0,3.0,0.0,Dried Sweetened Cranberries,117.0,19.0,nuts seeds dried fruit,snacks,9.0
1,676467,3,prior,10,3,16,17.0,1005.0,5.0,0.0,China Green Tips Green Tea,94.0,7.0,tea,beverages,107.0
2,2037211,3,prior,4,2,18,20.0,1819.0,1.0,0.0,All Natural No Stir Creamy Almond Butter,88.0,13.0,spreads,pantry,50.0
3,1972919,3,prior,6,0,16,7.0,1819.0,5.0,1.0,All Natural No Stir Creamy Almond Butter,88.0,13.0,spreads,pantry,69.0
4,1839752,3,prior,7,0,15,7.0,1819.0,2.0,1.0,All Natural No Stir Creamy Almond Butter,88.0,13.0,spreads,pantry,76.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7889713,414137,206205,prior,3,5,16,10.0,43352.0,3.0,1.0,Raspberries,32.0,4.0,packaged produce,produce,40.0
7889714,2658896,206205,prior,2,2,15,30.0,44471.0,4.0,0.0,Free & Clear Unscented Baby Wipes,82.0,18.0,baby accessories,babies,30.0
7889715,2658896,206205,prior,2,2,15,30.0,46820.0,7.0,0.0,Vanilla Pure Almond Milk,91.0,16.0,soy lactosefree,dairy eggs,30.0
7889716,969311,206205,prior,1,4,12,NaN,49235.0,4.0,0.0,Organic Half & Half,53.0,16.0,cream,dairy eggs,0.0


## days since last bought of this product

In [13]:
days_diff = sample_order_prior.groupby(['user_id', 'product_id'])['days_cumsum'].diff()
days_diff

0           NaN
1           NaN
2           NaN
3          19.0
4           7.0
           ... 
7889713    40.0
7889714     NaN
7889715     NaN
7889716     NaN
7889717     NaN
Name: days_cumsum, Length: 7889718, dtype: float64

In [14]:
sample_order_prior['days_last_bought_product'] = days_diff
sample_order_prior.head(30)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,days_cumsum,days_last_bought_product
0,444309,3,prior,2,3,19,9.0,248.0,3.0,0.0,Dried Sweetened Cranberries,117.0,19.0,nuts seeds dried fruit,snacks,9.0,NaN
1,676467,3,prior,10,3,16,17.0,1005.0,5.0,0.0,China Green Tips Green Tea,94.0,7.0,tea,beverages,107.0,NaN
2,2037211,3,prior,4,2,18,20.0,1819.0,1.0,0.0,All Natural No Stir Creamy Almond Butter,88.0,13.0,spreads,pantry,50.0,NaN
3,1972919,3,prior,6,0,16,7.0,1819.0,5.0,1.0,All Natural No Stir Creamy Almond Butter,88.0,13.0,spreads,pantry,69.0,19.0
4,1839752,3,prior,7,0,15,7.0,1819.0,2.0,1.0,All Natural No Stir Creamy Almond Butter,88.0,13.0,spreads,pantry,76.0,7.0
5,3002854,3,prior,3,3,16,21.0,7503.0,6.0,0.0,Whole Almonds,117.0,19.0,nuts seeds dried fruit,snacks,30.0,NaN
6,444309,3,prior,2,3,19,9.0,8021.0,5.0,0.0,100% Recycled Paper Towels,54.0,17.0,paper goods,household,9.0,NaN
7,1374495,3,prior,1,1,14,NaN,9387.0,1.0,0.0,Granny Smith Apples,24.0,4.0,fresh fruits,produce,0.0,NaN
8,2037211,3,prior,4,2,18,20.0,9387.0,3.0,1.0,Granny Smith Apples,24.0,4.0,fresh fruits,produce,50.0,50.0
9,2710558,3,prior,5,0,17,12.0,9387.0,2.0,1.0,Granny Smith Apples,24.0,4.0,fresh fruits,produce,62.0,12.0


In [28]:
order_diff = sample_order_prior.groupby(['user_id', 'product_id'])['order_number'].diff()
order_diff

0          NaN
1          NaN
2          NaN
3          2.0
4          1.0
          ... 
7889713    2.0
7889714    NaN
7889715    NaN
7889716    NaN
7889717    NaN
Name: order_number, Length: 7889718, dtype: float64

In [29]:
sample_order_prior['order_diff_last_bought_product'] = order_diff
sample_order_prior.head(30)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,days_cumsum,days_last_bought_product,order_diff_last_bought_product
0,444309,3,prior,2,3,19,9.0,248.0,3.0,0.0,Dried Sweetened Cranberries,117.0,19.0,nuts seeds dried fruit,snacks,9.0,NaN,NaN
1,676467,3,prior,10,3,16,17.0,1005.0,5.0,0.0,China Green Tips Green Tea,94.0,7.0,tea,beverages,107.0,NaN,NaN
2,2037211,3,prior,4,2,18,20.0,1819.0,1.0,0.0,All Natural No Stir Creamy Almond Butter,88.0,13.0,spreads,pantry,50.0,NaN,NaN
3,1972919,3,prior,6,0,16,7.0,1819.0,5.0,1.0,All Natural No Stir Creamy Almond Butter,88.0,13.0,spreads,pantry,69.0,19.0,2.0
4,1839752,3,prior,7,0,15,7.0,1819.0,2.0,1.0,All Natural No Stir Creamy Almond Butter,88.0,13.0,spreads,pantry,76.0,7.0,1.0
5,3002854,3,prior,3,3,16,21.0,7503.0,6.0,0.0,Whole Almonds,117.0,19.0,nuts seeds dried fruit,snacks,30.0,NaN,NaN
6,444309,3,prior,2,3,19,9.0,8021.0,5.0,0.0,100% Recycled Paper Towels,54.0,17.0,paper goods,household,9.0,NaN,NaN
7,1374495,3,prior,1,1,14,NaN,9387.0,1.0,0.0,Granny Smith Apples,24.0,4.0,fresh fruits,produce,0.0,NaN,NaN
8,2037211,3,prior,4,2,18,20.0,9387.0,3.0,1.0,Granny Smith Apples,24.0,4.0,fresh fruits,produce,50.0,50.0,3.0
9,2710558,3,prior,5,0,17,12.0,9387.0,2.0,1.0,Granny Smith Apples,24.0,4.0,fresh fruits,produce,62.0,12.0,1.0


In [30]:
sample_order_prior.to_csv("instacart/sample_order_prior_diff.csv", index=False)

In [17]:
sample_order_train.to_csv("instacart/sample_order_train_cumsum.csv", index=False)
sample_order_test.to_csv("instacart/sample_order_test_cumsum.csv", index=False)

## Aggregate features for user x product

In [36]:
sample_order_prior['order_hour_of_day'].mode()[0]

10

In [37]:
sample_order_prior_user_product = sample_order_prior.groupby(['user_id', 'product_id']).agg(
    days_cumsum=('days_cumsum', 'last'),
    num_prod_bought=('order_number', 'count'),
    days_since_last_bought=('days_last_bought_product', 'last'),
    average_days_period=('days_last_bought_product', 'mean'),
    orders_since_last_bought=('order_diff_last_bought_product', 'last'),
    average_orders_period=('order_diff_last_bought_product', 'mean'),
    dow_usually_bought=('order_dow', lambda x: x.mode()[0]),
    hod_usually_bought=('order_hour_of_day', lambda x: x.mode()[0]),
    department_id=('department_id', 'last'),
    aisle_id=('aisle_id', 'last'))

In [38]:
sample_order_prior_user_product

days_cumsum  num_prod_bought  days_since_last_bought  \
user_id product_id                                                         
3       248.0               9.0                1                     NaN   
        1005.0            107.0                1                     NaN   
        1819.0             76.0                3                     7.0   
        7503.0             30.0                1                     NaN   
        8021.0              9.0                1                     NaN   
...                         ...              ...                     ...   
206205  43352.0            40.0                2                    40.0   
        44471.0            30.0                1                     NaN   
        46820.0            30.0                1                     NaN   
        49235.0             0.0                1                     NaN   
        49628.0             0.0                1                     NaN   

                    average_days_period  orders_since_last_bought  \
user_id product_id                                                  
3       248.0                       NaN                       NaN   
        1005.0                      NaN                       NaN   
        1819.0                     13.0                       1.0   
        7503.0                      NaN                       NaN   
        8021.0                      NaN                       NaN   
...                                 ...                       ...   
206205  43352.0                    40.0                       2.0   
        44471.0                     NaN                       NaN   
        46820.0                     NaN                       NaN   
        49235.0                     NaN                       NaN   
        49628.0                     NaN                       NaN   

                    average_orders_period  dow_usually_bought  \
user_id product_id                                              
3       248.0                         NaN                   3   
        1005.0                        NaN                   3   
        1819.0                        1.5                   0   
        7503.0                        NaN                   3   
        8021.0                        NaN                   3   
...                                   ...                 ...   
206205  43352.0                       2.0                   4   
        44471.0                       NaN                   2   
        46820.0                       NaN                   2   
        49235.0                       NaN                   4   
        49628.0                       NaN                   4   

                    hod_usually_bought  department_id  aisle_id  
user_id product_id                                               
3       248.0                       19           19.0     117.0  
        1005.0                      16            7.0      94.0  
        1819.0                      15           13.0      88.0  
        7503.0                      16           19.0     117.0  
        8021.0                      19           17.0      54.0  
...                                ...            ...       ...  
206205  43352.0                     12            4.0      32.0  
        44471.0                     15           18.0      82.0  
        46820.0                     15           16.0      91.0  
        49235.0                     12           16.0      53.0  
        49628.0                     12           16.0     120.0  

[3231752 rows x 10 columns]

In [39]:
sample_order_prior_user_product.to_csv("instacart/sample_order_prior_user_product.csv")

## Aggregate features for user x department

In [40]:
sample_order_prior['department_id'].nunique()

21

In [43]:
sample_order_prior_user_department = sample_order_prior.groupby(['user_id', 'department_id']).agg(
    num_prod_bought_dep=('add_to_cart_order', 'count'),
    num_order_bought_dep=('order_number', 'nunique')
)
sample_order_prior_user_department

num_prod_bought_dep  num_order_bought_dep
user_id department_id                                           
3       1.0                              6                     5
        4.0                             38                    12
        7.0                              3                     3
        9.0                              4                     4
        13.0                             4                     4
...                                    ...                   ...
206205  12.0                             1                     1
        16.0                            14                     3
        18.0                             1                     1
        20.0                             3                     1
        21.0                             1                     1

[541778 rows x 2 columns]

In [55]:
sample_order_prior_user_department.to_csv("instacart/sample_order_prior_user_department.csv")

## Aggregate features for user x aisle

In [44]:
sample_order_prior_user_aisle = sample_order_prior.groupby(['user_id', 'aisle_id']).agg(
    num_prod_bought_ais=('add_to_cart_order', 'count'),
    num_order_bought_ais=('order_number', 'nunique')
)
sample_order_prior_user_aisle

num_prod_bought_ais  num_order_bought_ais
user_id aisle_id                                           
3       4.0                         4                     4
        13.0                        2                     2
        21.0                        5                     4
        24.0                       19                    12
        37.0                        4                     4
...                               ...                   ...
206205  98.0                        1                     1
        100.0                       1                     1
        112.0                       2                     2
        116.0                       1                     1
        120.0                       5                     2

[1390425 rows x 2 columns]

In [56]:
sample_order_prior_user_aisle.to_csv("instacart/sample_order_prior_user_aisle.csv")

## Aggregate features for user

In [47]:
sample_order_prior.columns

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'aisle', 'department', 'days_cumsum',
       'days_last_bought_product', 'order_diff_last_bought_product'],
      dtype='object')

In [52]:
sample_order_prior_user = sample_order_prior[['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order', 'days_cumsum']].sort_values(
    by=['user_id', 'order_number']
).drop_duplicates().reset_index(drop=True)

sample_order_prior_user

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,days_cumsum
0,1374495,3,prior,1,1,14,NaN,0.0
1,444309,3,prior,2,3,19,9.0,9.0
2,3002854,3,prior,3,3,16,21.0,30.0
3,2037211,3,prior,4,2,18,20.0,50.0
4,2710558,3,prior,5,0,17,12.0,62.0
...,...,...,...,...,...,...,...,...
780591,1869536,206202,prior,21,4,0,30.0,263.0
780592,1112560,206202,prior,22,3,14,6.0,269.0
780593,969311,206205,prior,1,4,12,NaN,0.0
780594,2658896,206205,prior,2,2,15,30.0,30.0


In [54]:
sample_order_prior_user = sample_order_prior_user.groupby(['user_id']).agg(
    number_user_order=('order_number', max),
    dow_user_usually_bought=('order_dow', lambda x: x.mode()[0]),
    hod_user_usually_bought=('order_hour_of_day', lambda x: x.mode()[0]),
    days_cumsum_last=('days_cumsum', 'last'),
    average_daydiff_user=('days_since_prior_order', 'mean')
)
sample_order_prior_user

,number_user_order,dow_user_usually_bought,hod_user_usually_bought,days_cumsum_last,average_daydiff_user
user_id,,,,,
3,12,0,16,133.0,12.090909
5,4,3,18,40.0,13.333333
13,12,6,12,84.0,7.636364
25,3,1,14,60.0,30.000000
29,18,5,11,196.0,11.529412
...,...,...,...,...,...
206189,6,0,15,80.0,16.000000
206191,5,0,9,78.0,19.500000
206195,19,6,9,122.0,6.777778


In [57]:
sample_order_prior_user.to_csv("instacart/sample_order_prior_user.csv")